In [1]:
ktag="hdp"

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
## load star networks

In [4]:
topics=pd.read_pickle("/data/Hoaxy/topic_cluster_%s.pkl.gz"%ktag)[['doc_id','topic_id']]

In [5]:
print("# of topics: %d"%topics["topic_id"].nunique())

# of topics: 44


In [6]:
topics.head()

,doc_id,topic_id
0,1099843,0
1,1099884,6
2,1100440,0
3,1099777,0
4,1099632,0


In [7]:
doc_topics_size=topics.groupby("topic_id").size().reset_index(name="# of documents")

In [8]:
doc_topics_size

,topic_id,# of documents
0,0,11957
1,1,939
2,2,272
3,3,163
4,4,594
5,5,138
6,6,26
7,7,3
8,8,11
9,9,21


In [9]:
doc_topics=topics.groupby("topic_id")["doc_id"].apply(list)

In [10]:
doc_topics.head()

topic_id
0    [1099843, 1100440, 1099777, 1099632, 1099458, ...
1    [1099451, 1099061, 1100102, 1099295, 1099736, ...
2    [1099493, 1099393, 1099385, 1099516, 1099762, ...
3    [1099297, 1099619, 1099747, 1099765, 1099401, ...
4    [1099140, 1099138, 1099139, 1099131, 1099135, ...
Name: doc_id, dtype: object

In [11]:
##doc_topics.loc[0]

In [12]:
import random
import itertools
 
def pair_generator(numbers):
  # Keep track of already generated pairs
  used_pairs = set()
 
  while True:
    pair = random.sample(numbers, 2)
    # Avoid generating both (1, 2) and (2, 1)
    pair = tuple(sorted(pair))
    if pair not in used_pairs:
      used_pairs.add(pair)
      yield pair

In [13]:
## prepare records in positive classes
def _get_pairs(topic_id):
    doc_list=list(doc_topics.loc[topic_id])
    gen = pair_generator(doc_list)
 
    # Get 50000 pairs:
    N=50000
    
    no_docs=len(doc_list)
    no_docs=int(no_docs*(no_docs-1)/2)
    N=min(N,no_docs)
    for i in range(N):
        pair = gen.__next__()
        ##print(pair)
        record=sorted(list(pair))
        record.append(1)
        positives.append(record)

In [14]:
positives=[]
doc_topics_size["topic_id"].apply(_get_pairs)

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25    None
26    None
27    None
28    None
29    None
30    None
31    None
32    None
33    None
34    None
35    None
36    None
37    None
38    None
39    None
40    None
41    None
42    None
43    None
Name: topic_id, dtype: object

In [15]:
print("# positives: %d"%len(positives))

# positives: 265768


In [16]:
def neg_pair_generator():
  # Keep track of already generated pairs
  used_pairs = set()
 
  while True:
    neg_pair=topics.sample(2)
    no_topics=neg_pair["doc_id"].nunique()
    ###print(no_topics)
    if no_topics>1:
        pair=tuple(sorted(neg_pair["doc_id"].values))
        if pair not in used_pairs:
            used_pairs.add(pair)
            yield pair

In [17]:
negatives=[]
neg_gen = neg_pair_generator()
N=len(positives)
for i in range(N):
    pair = neg_gen.__next__()
    ##print(pair)
    record=sorted(list(pair))
    record.append(0)
    negatives.append(record)

In [18]:
print("# negatives: %d"%len(negatives))

# negatives: 265768


In [19]:
examples=pd.DataFrame(positives+negatives,columns=['document1','document2','class'])

In [20]:
examples.tail()

,document1,document2,class
531531,1101434,1102740,0
531532,1100917,1128309,0
531533,1108198,1123413,0
531534,1103588,1124164,0
531535,1109055,1115580,0


In [21]:
examples.to_pickle("/data/Hoaxy/%s_doc_pairs.pkl.gz"%ktag)